<a href="https://colab.research.google.com/github/Samin-Sadaf7/Langchain_Fundamentals/blob/main/ConversationEntityMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### ConversationEntityMemory is a memory class provided by LangChain, designed to track and store information about entities that arise in a conversation. It allows the AI to "remember" key facts about people, places, or concepts mentioned during a conversation, so it can refer back to them later on, improving the conversational experience.

## Key Features:
**Entity Tracking:** It identifies entities (e.g., names, places, concepts) and stores relevant information about them. For instance, if you mention "Tanmay" in one part of a conversation, it can remember details about "Tanmay" for later reference.

**Context-Aware:** It helps the AI maintain context by remembering details about the entities mentioned during the chat, ensuring more natural, fluid conversations over time.

**Customization:** You can customize what to store and how to retrieve it during future interactions.

In [18]:
!pip install langchain

In [19]:
!pip install -U langchain-community

In [20]:
!pip install langchain_google_genai

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GeminiKey')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [24]:
model = ChatGoogleGenerativeAI(model="gemini-1.0-pro",convert_system_message_to_human=True)

In [25]:
print(model.invoke("hi").content)

Hello there! How may I assist you today?


In [26]:
from langchain.memory import ConversationEntityMemory

In [27]:
memory = ConversationEntityMemory(llm=model)

In [28]:
_input= {"input": "i am very hungry."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {}}

In [29]:
_input= {"input": "sunny & mayank are working on a hackathon project"}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Sunny': '', 'Mayank': ''}}

In [30]:
_input= {"input": "My name is John, and I'm planning a trip to Paris."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'John': '', 'Paris': ''}}

In [31]:
_input= {"input": "Sunny is a great person who values gratitude."}
memory.load_memory_variables(_input)

{'history': '', 'entities': {'Sunny': ''}}

In [32]:
memory.save_context(
    {"Human": "Sunny and Mayank are working on a hackathon project"},
    {"AI": "That's awesome! What's the hackathon project about?"}
)

In [33]:
memory.load_memory_variables({"input": "who is Sunny?"})

{'history': "Human: Sunny and Mayank are working on a hackathon project\nAI: That's awesome! What's the hackathon project about?",
 'entities': {}}

In [34]:
memory.save_context(
    {"Human": "It's a machine learning project focused on healthcare."},
     {"AI": "Sounds exciting! Are they building a prediction model or something else?"}
)

In [35]:
memory.save_context(
    {"Human": "Yes, they are building prediction model."},
    {"AI": "Wishing Sunny and Mayank all the best for their project!"}
)

In [36]:
memory.load_memory_variables({"input": "who is Sunny?"})

{'history': "Human: Sunny and Mayank are working on a hackathon project\nAI: That's awesome! What's the hackathon project about?\nHuman: It's a machine learning project focused on healthcare.\nAI: Sounds exciting! Are they building a prediction model or something else?\nHuman: Yes, they are building prediction model.\nAI: Wishing Sunny and Mayank all the best for their project!",
 'entities': {'Sunny': 'Sunny is working on a hackathon project with Mayank.'}}

In [37]:
print('You are an AI assistant helping a human keep track of facts about relevant people, places, and concepts in their life. Update the summary of the provided entity in the "Entity" section based on the last line of your conversation with the human. If you are writing the summary for the first time, return a single sentence.\nThe update should only include facts that are relayed in the last line of conversation about the provided entity, and should only contain facts about the provided entity.\n\nIf there is no new information about the provided entity or the information is not worth noting (not an important or relevant fact to remember long-term), return the existing summary unchanged.\n\nFull conversation history (for context):\n{history}\n\nEntity to summarize:\n{entity}\n\nExisting summary of {entity}:\n{summary}\n\nLast line of conversation:\nHuman: {input}\nUpdated summary:')

You are an AI assistant helping a human keep track of facts about relevant people, places, and concepts in their life. Update the summary of the provided entity in the "Entity" section based on the last line of your conversation with the human. If you are writing the summary for the first time, return a single sentence.
The update should only include facts that are relayed in the last line of conversation about the provided entity, and should only contain facts about the provided entity.

If there is no new information about the provided entity or the information is not worth noting (not an important or relevant fact to remember long-term), return the existing summary unchanged.

Full conversation history (for context):
{history}

Entity to summarize:
{entity}

Existing summary of {entity}:
{summary}

Last line of conversation:
Human: {input}
Updated summary:


In [38]:
print('You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.\n\nThe conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.\n\nReturn the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).\n\nEXAMPLE\nConversation history:\nPerson #1: how\'s it going today?\nAI: "It\'s going great! How about you?"\nPerson #1: good! busy working on Langchain. lots to do.\nAI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"\nLast line:\nPerson #1: i\'m trying to improve Langchain\'s interfaces, the UX, its integrations with various products the user might want ... a lot of stuff.\nOutput: Langchain\nEND OF EXAMPLE\n\nEXAMPLE\nConversation history:\nPerson #1: how\'s it going today?\nAI: "It\'s going great! How about you?"\nPerson #1: good! busy working on Langchain. lots to do.\nAI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"\nLast line:\nPerson #1: i\'m trying to improve Langchain\'s interfaces, the UX, its integrations with various products the user might want ... a lot of stuff. I\'m working with Person #2.\nOutput: Langchain, Person #2\nEND OF EXAMPLE\n\nConversation history (for reference only):\n{history}\nLast line of conversation (for extraction):\nHuman: {input}\n\nOutput:')

You are an AI assistant reading the transcript of a conversation between an AI and a human. Extract all of the proper nouns from the last line of conversation. As a guideline, a proper noun is generally capitalized. You should definitely extract all names and places.

The conversation history is provided just in case of a coreference (e.g. "What do you know about him" where "him" is defined in a previous line) -- ignore items mentioned there that are not in the last line.

Return the output as a single comma-separated list, or NONE if there is nothing of note to return (e.g. the user is just issuing a greeting or having a simple conversation).

EXAMPLE
Conversation history:
Person #1: how's it going today?
AI: "It's going great! How about you?"
Person #1: good! busy working on Langchain. lots to do.
AI: "That sounds like a lot of work! What kind of things are you doing to make Langchain better?"
Last line:
Person #1: i'm trying to improve Langchain's interfaces, the UX, its integration

In [39]:
from langchain.chains import ConversationChain

In [40]:
from langchain.memory import ConversationEntityMemory

In [41]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [42]:
from pydantic import BaseModel

In [43]:
from typing import List, Dict, Any

In [44]:
conversation = ConversationChain(
    llm=model,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=model)
)

In [45]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's great! Hackathons can be a lot of fun and a great way to learn new skills. What kind of project are they working on?"

In [46]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [47]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's really interesting! Langchain is a very powerful tool, and adding more complex memory structures could open up a lot of new possibilities. What kind of memory structures are they trying to add?"

In [50]:
conversation.predict(input="They are adding in a key-value store for entities mentioned so far in the conversation.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'That sounds like a very useful addition! A key-value store can be a very efficient way to store and retrieve data, and it could be very helpful for keeping track of entities in a conversation. This could be especially useful for tasks like question answering and dialogue generation, where it is important to be able to keep track of the entities that have been mentioned in the conversation so far.'

In [51]:
conversation.predict(input="What do you know about Deven & Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I know that Deven and Sam are working on a hackathon project together. They are trying to add more complex memory structures to Langchain, specifically a key-value store for entities mentioned so far in the conversation. This could be a very useful addition, as it could help with tasks like question answering and dialogue generation.'

In [52]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Deven': 'Deven is working on a hackathon project with Sam, specifically to '
          'add more complex memory structures to Langchain, including a '
          'key-value store for entities mentioned so far in the conversation.',
 'Langchain': 'Langchain is a powerful tool with the potential for new '
              'possibilities through the addition of more complex memory '
              'structures.',
 'Sam': 'Sam is working on a hackathon project with Deven, specifically to add '
        'a key-value store for entities mentioned in the conversation to '
        'Langchain.'}


In [53]:
conversation.predict(input="Sam is the founder of a company called Daimon.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"That's interesting! I didn't know that. I'm not familiar with Daimon, but I'm always interested in learning about new companies. What does Daimon do?"

In [54]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Daimon': 'Daimon is a company founded by Sam.',
 'Deven': 'Deven is working on a hackathon project with Sam, specifically to '
          'add more complex memory structures to Langchain, including a '
          'key-value store for entities mentioned so far in the conversation.',
 'Langchain': 'Langchain is a powerful tool with the potential for new '
              'possibilities through the addition of more complex memory '
              'structures.',
 'Sam': 'Sam is working on a hackathon project with Deven, specifically to add '
        'a key-value store for entities mentioned in the conversation to '
        'Langchain. Sam is also the founder of a company called Daimon.'}


In [55]:
conversation.predict(input="What do you know about Sam?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'I know that Sam is working on a hackathon project with Deven, specifically to add a key-value store for entities mentioned in the conversation to Langchain. Sam is also the founder of a company called Daimon.'